In [ ]:
import os
import keras_tuner

import tensorflow as tf
from tensorflow import keras

In [ ]:
"GPU available" if tf.config.list_physical_devices("GPU") else "Not available"

In [ ]:
image_size = (50, 50)
batch_size = 512
class_names = ["idc", "non_idc"]
num_classes = len(class_names)

In [ ]:
train_set = keras.utils.image_dataset_from_directory(
    directory="../data/dataset/train",
    label_mode="binary",
    batch_size=batch_size,
    image_size=image_size,
    class_names=class_names
)

val_set = keras.utils.image_dataset_from_directory(
    directory="../data/dataset/val",
    label_mode="binary",
    batch_size=batch_size,
    image_size=image_size,
    class_names=class_names
)

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=image_size + (3,))

    x = keras.layers.Rescaling(1.0 / 255)(inputs)
    
    x = keras.layers.Conv2D(
        filters=hp.Int(name="filters1", min_value=32, max_value=256, step=32), 
        kernel_size=hp.Choice(name="kernel_size1", values=[3, 5]), 
        activation="relu"
        )(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = keras.layers.Conv2D(
        filters=hp.Int(name="filters2", min_value=32, max_value=256, step=32), 
        kernel_size=hp.Choice(name="kernel_size2", values=[3, 5]), 
        activation="relu"
        )(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = keras.layers.Conv2D(
        filters=hp.Int(name="filters3", min_value=32, max_value=256, step=32), 
        kernel_size=hp.Choice(name="kernel_size3", values=[3, 5]), 
        activation="relu"
        )(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(
        units=hp.Int(name="units", min_value=32, max_value=256, step=32), 
        activation="relu"
        )(x)
    
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(units, activation=activation)(x)

    model = keras.Model(inputs, outputs)

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [ ]:
tuner = keras_tuner.tuners.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory=f".",
    project_name=".",
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(
    train_set,
    epochs=100,
    validation_data=val_set,
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)

In [ ]:
tuner.results_summary()

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]

best_model.build(input_shape=(None, 50, 50, 3))
best_model.summary()